About this notebook:

Provided dataset train set will be used for train and validation. Test set can only be used for evaluation. 

Samples in train set do not all contains long answer (further then nor short answer). 

Output: wlong_nq_train.json
Notes: this output filtered out samples containing long answer. 

In [ ]:
### Import Libraries and Dependencies
!pip install -q transformers
!pip install datasets
!pip install apache_beam

from datasets import load_dataset
from collections import Counter
import numpy as np
import tensorflow as tf
from tensorflow import keras
import transformers

     |████████████████████████████████| 4.4 MB 7.4 MB/s 
     |████████████████████████████████| 596 kB 37.5 MB/s 
     |████████████████████████████████| 101 kB 4.6 MB/s 
     |████████████████████████████████| 6.6 MB 48.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 9.0 MB/s 
     |████████████████████████████████| 212 kB 54.1 MB/s 
     |████████████████████████████████| 140 kB 45.6 MB/s 
     |████████████████████████████████| 1.1 MB 42.0 MB/s 
     |████████████████████████████████| 127 kB 56.4 MB/s 
     |████████████████████████████████| 144 kB 58.1 MB/s 
     |████████████████████████████████| 271 kB 43.6 MB/s 
     |████████████████████████████████| 94 kB 1.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently

### Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd drive/MyDrive/w266/data/natural_questions/

In [ ]:
!ls -lh drive/MyDrive/w266/data/natural_questions/

total 22G
-rw------- 1 root root   18M Oct 28  2019 simplified-nq-test.jsonl
-rw------- 1 root root   17G Oct 28  2019 simplified-nq-train.jsonl
-rw------- 1 root root 1019M Jun 28 02:14 v1.0-simplified_nq-dev-all.jsonl.gz
-rw------- 1 root root  4.4G Jun 28 00:47 v1.0-simplified_simplified-nq-train.jsonl.gz


In [ ]:
path = '/drive/MyDrive/w266/data/natural_questions/'

In [ ]:
tf.__version__

'2.8.2'

In [ ]:
print(transformers.__version__)

4.20.1


Download Natural Question dataset

In [ ]:
from datasets import load_dataset

train_set = load_dataset('json', data_files='drive/MyDrive/w266/data/natural_questions/simplified-nq-train.jsonl')['train']
test_set = load_dataset('json', data_files='drive/MyDrive/w266/data/natural_questions/simplified-nq-test.jsonl')['test']

Using custom data configuration default-b0c6ea5bffe851d8


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b0c6ea5bffe851d8/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# num of samples in train_set
len(train_set) 

307373

In [ ]:
# num of samples in test_set
len(test_set) 

346

In [ ]:
type(train_set[0])

dict

In [ ]:
train_set[0].keys()

dict_keys(['document_text', 'long_answer_candidates', 'question_text', 'annotations', 'document_url', 'example_id'])

In [ ]:
train_set[0]

{'annotations': [{'annotation_id': 5.931654502200276e+17,
   'long_answer': {'candidate_index': 54,
    'end_token': 2019,
    'start_token': 1952},
   'short_answers': [{'end_token': 1969, 'start_token': 1960}],
   'yes_no_answer': 'NONE'}],
 'document_text': "Email marketing - Wikipedia <H1> Email marketing </H1> Jump to : navigation , search <Table> <Tr> <Td> </Td> <Td> ( hide ) This article has multiple issues . Please help improve it or discuss these issues on the talk page . ( Learn how and when to remove these template messages ) <Table> <Tr> <Td> </Td> <Td> This article needs additional citations for verification . Please help improve this article by adding citations to reliable sources . Unsourced material may be challenged and removed . ( September 2014 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table> <Tr> <Td> </Td> <Td> This article possibly contains original research . Please improve it by verifying the claims made and adding inline cit

In [ ]:
test_set[0].keys()

dict_keys(['example_id', 'question_text', 'document_text', 'long_answer_candidates'])

In [ ]:
train_set[0].keys()

dict_keys(['document_text', 'long_answer_candidates', 'question_text', 'annotations', 'document_url', 'example_id'])

In [ ]:
test_set[0]

In [ ]:
# sample_id = 0
long_ans =0
no_long_ans = 0
for i in range(len(train_set)):
  start_token = train_set[i]['annotations'][0]['long_answer']['start_token'] 
  end_token = train_set[i]['annotations'][0]['long_answer']['end_token']
  
  if start_token == -1 and end_token == -1:
    no_long_ans += 1
  else: 
    long_ans += 1
print('Number of samples in training set that contains long answer:'+ str(long_ans))
print('Number of samples in training set that do not contain long answer:'+ str(no_long_ans))

Number of samples in training set that contains long answer:152148
Number of samples in training set that do not contain long answer:155225


Remove the samples that do not have a long answer (with annotation in start_token and end_token equal to -1).

In [ ]:
wlong_train = train_set.filter(lambda example: example['annotations'][0]['long_answer']['start_token'] != -1 and example['annotations'][0]['long_answer']['end_token'] != -1)
len(wlong_train)

  0%|          | 0/308 [00:00<?, ?ba/s]

152148

In [ ]:
wlong_train.save_to_disk("drive/MyDrive/w266/data/natural_questions/wlong_nq_train.json")

Flattening the indices:   0%|          | 0/153 [00:00<?, ?ba/s]